In [1]:
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('ANOVA data.csv')
data = data[['A','B','n','y']]
data.head()

,A,B,n,y
0,a1,b1,1,38.120639
1,a2,b1,1,55.550930
2,a1,b2,1,42.493114
3,a2,b2,1,49.785842
4,a1,b3,1,40.988523


In [3]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

In [4]:
formula = 'y ~ C(A) + C(B) + C(A):C(B)'
model = ols(formula,data).fit()
aov_table = anova_lm(model, typ=2)
aov_table = eta_squared(aov_table)
aov_table = omega_squared(aov_table)
print(aov_table)

                sum_sq    df           F        PR(>F)    eta_sq  omega_sq
C(A)       2352.676204   1.0  348.346480  3.267824e-25  0.486960  0.484885
C(B)        510.821829   2.0   37.817143  5.385112e-11  0.105731  0.102791
C(A):C(B)  1603.144818   2.0  118.683959  1.716741e-20  0.331821  0.328566
Residual    364.707331  54.0         NaN           NaN       NaN       NaN
